In [3]:
import numpy as np
import pandas as pd

from btopt.data.dataloader import YFDataloader
from btopt.data.dataview import DataView, DataViewNumpy
from btopt.data.timeframe import Timeframe
from btopt.engine import Engine


class SimpleEngine(Engine):
    def __init__(self, optimized_data: DataViewNumpy):
        super().__init__(optimized_data)
        self.total_trades = 0
        self.profitable_trades = 0

    def _process_data_point(
        self,
        symbol: str,
        timeframe: Timeframe,
        timestamp: pd.Timestamp,
        ohlcv_data: np.ndarray,
    ):
        open_price, high_price, low_price, close_price, volume, _ = ohlcv_data

        # Simple strategy: Buy if the close is higher than the open, sell otherwise
        if close_price > open_price:
            self.total_trades += 1
            if high_price > close_price * 1.01:  # 1% profit target
                self.profitable_trades += 1

        # Print progress every 1000 data points
        if self.total_trades % 1000 == 0 and self.total_trades > 0:
            win_rate = (self.profitable_trades / self.total_trades) * 100
            print(
                f"Processed {self.total_trades} trades. Current win rate: {win_rate:.2f}%"
            )


def main():
    # Load data for multiple symbols and timeframes
    symbols = ["EURUSD", "GBPUSD", "USDJPY"]
    timeframes = ["1m", "5m", "15m", "1h", "4h", "1d"]

    # Initialize CSVDataLoader
    csv_loader = YFDataloader(
        [
            "EURUSD",
        ],
        "1w",
    )

    # Create MarketDataRepository instance
    market_data = DataView()

    for symbol in symbols:
        for tf in timeframes:
            df = csv_loader._load_data(symbol, tf)
            if df is not None:
                market_data.add_data(symbol, tf, df)

    # Align the data
    market_data.align_all_data()

    # Create OptimizedDataView
    optimized_data = DataViewNumpy(market_data)

    # Create and run the engine
    engine = SimpleEngine(optimized_data)
    engine.run()

    # Print final results
    total_trades = engine.total_trades
    profitable_trades = engine.profitable_trades
    win_rate = (profitable_trades / total_trades) * 100 if total_trades > 0 else 0

    print("\nBacktest completed!")
    print(f"Total trades: {total_trades}")
    print(f"Profitable trades: {profitable_trades}")
    print(f"Final win rate: {win_rate:.2f}%")

    # Print data range
    start_date, end_date = optimized_data.get_data_range()
    print(f"\nData range: from {start_date} to {end_date}")

    # Print some statistics
    print("\nSymbols:", optimized_data.symbols)
    print("Timeframes:", [str(tf) for tf in optimized_data.timeframes])


if __name__ == "__main__":
    main()